In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

# Load data

In [49]:
# python load data
from utils.pre_processing import combine_scraped_data
from config import DATA_PATHS
from utils.pre_processing import clean_scraped_data
from utils.pre_processing import labels_indexes_mapping
from sklearn.model_selection import train_test_split

for phase in DATA_PATHS:
    metadata_name = DATA_PATHS[phase]['metadata']
    data_name = DATA_PATHS[phase]['data']
    combine_scraped_data(f'data/{data_name}', f'data/{metadata_name}', is_train=False)

# TRAIN
train = clean_scraped_data('data/train_processed.xlsx')

# TRAIN , VAL
train, val = train_test_split(train, test_size=0.25, random_state=11)

# TEST
test = clean_scraped_data('data/test_processed.xlsx')


# Create labels to indexes and indexes to labels dicts
label_to_idx, idx_to_label = labels_indexes_mapping(train)
# change the label to number
train.loc[:, 'labels'] = train.labels.apply(lambda x: label_to_idx.get(x))

In [50]:
# convert the data to pytorch dataset

from datasets import TitlesDataset

train_ds = TitlesDataset(train.title.tolist(), train.labels.tolist())
val_ds = TitlesDataset(val.title.tolist(), val.labels.tolist())
test_ds = TitlesDataset(test.title.tolist(), test.labels.tolist())

# BERT

In [27]:
import torch
# PARAMETERS

# train parameters
batch_size = 8
num_epochs = 15
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'DEVICE: {device}')

# embedding parameters
max_length = 128

#optimizer parameters
lr = 5e-5

DEVICE: cuda


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds)
val_dataloader = DataLoader(val_ds)

In [29]:
from models import BERT
from transformers import AdamW

model = BERT(model_name='bert-base-uncased',num_labels=len(label_to_idx), max_length=max_length,device=device)

optimizer = AdamW(model.parameters(), lr=lr)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
from trainer import Trainer
trainer = Trainer(model=model, optimizer=optimizer, train_dataloader=train_dataloader,
                  num_epochs=num_epochs, device=device)

In [30]:
trainer.train()

  0%|          | 0/9665 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [31]:
labels, preds = trainer.evaluate(train_dataloader)

  0%|          | 0/1933 [00:00<?, ?it/s]

# tBERT

In [6]:
import torch
from models import tBERT
from transformers import AdamW

# PARAMETERS

# train parameters
batch_size = 8
num_epochs = 5
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# lda parameters
n_topics = 70

# embedding parameters
max_length = 64

#optimizer parameters
lr = 5e-5

In [47]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds)
val_dataloader = DataLoader(val_ds)

NameError: name 'val_ds' is not defined

In [8]:
from utils.pre_processing import create_titles_corpus

corpus = create_titles_corpus(train_dataloader)[:10]
model = tBERT(corpus, model_name='bert-base-uncased',num_labels=len(label_to_idx), n_topics=n_topics,
             max_length=max_length, device=device)

optimizer = AdamW(model.parameters(), lr=lr)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from trainer import Trainer
trainer = Trainer(model=model, optimizer=optimizer, train_dataloader=train_dataloader, num_epochs=num_epochs, device=device)

In [12]:
trainer.train()

  0%|          | 0/9665 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
labels, preds = trainer.evaluate(train_dataloader)

  0%|          | 0/1933 [00:00<?, ?it/s]